In [1]:
import pandas as pd
# https://www.avera.org/patients-visitors/price-transparency/

In [2]:
import csv
import subprocess
from urllib.parse import urlparse
import os

from dateutil.parser import parse as parse_datetime
import pandas as pd

In [3]:
TARGET_COLUMNS = [ 
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [4]:
ccn = "241343"
url = "https://www.avera.org/app/files/public/84e6c5b3-a05c-41ab-bc99-9e7cdecc6698/843156881_avera-granite-falls-health-center_standardcharges.xlsx"
filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [5]:
import httpx
import os

def create_session():
    session = httpx.Client(http2=True)
    
    session.headers = {
        'authority': 'www.avera.org',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }
    
    resp = session.get("https://www.avera.org/patients-visitors/price-transparency/")
    print(resp.url)
    
    return session

def download_file(session, url, filename):
    if os.path.isfile(filename):
        print("{} already downloaded - skipping".format(filename))
        return
    
    resp = session.get(url)
    print(resp.url)
    
    out_f = open(filename, "wb")
    out_f.write(resp.content)
    out_f.close()
    
session = create_session()
download_file(session, url, filename)

https://www.avera.org/patients-visitors/price-transparency/
843156881_avera-granite-falls-health-center_standardcharges.xlsx already downloaded - skipping


In [6]:
df_in = pd.read_excel(filename, skiprows=1, sheet_name='Service Bundle')
df_in

,Facility Name,Item / Service Description,Gross Charge,Minimum Expected Payment,Maximum Expected Payment,Avera Health Plan Commeercial Expected Payment,BCBSMN AWA Expected Payment,HealthPartners Commercial Expected Payment,Medica Expected Commercial Payment,PreferredOne PPO Expected Payment,United Healthcare Commercial Expected Payment,Discounted Cash Price
0,Granite Falls Health Center,Tobacco Use Cessation Intensive >10 Minutes,NaN,37.7270,37.7270,NaN,37.7270,NaN,NaN,NaN,NaN,NaN
1,Granite Falls Health Center,Tobacco Use Cessation Intermediate 3-10 Minutes,NaN,19.3502,19.3502,NaN,19.3502,NaN,NaN,NaN,NaN,NaN
2,Granite Falls Health Center,Prolng E/M Before&/After Dir Care Ea 30 Minutes,NaN,171.1820,171.1820,NaN,171.1820,NaN,NaN,NaN,NaN,NaN
3,Granite Falls Health Center,Prolng E/M Svc Before&/After Dir Pt Care 1St Hr,NaN,332.9958,332.9958,NaN,332.9958,NaN,NaN,NaN,NaN,NaN
4,Granite Falls Health Center,Nursing Facility Discharge Management 30 Minutes,NaN,307.8300,307.8300,NaN,307.8300,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
990,Granite Falls Health Center,Incision & Removal Foreign Body Subq Tiss Compl,NaN,438.0300,438.0300,NaN,438.0300,NaN,NaN,NaN,NaN,NaN
991,Granite Falls Health Center,Incision & Removal Foreign Body Subq Tiss Simple,NaN,242.4138,242.4138,NaN,242.4138,NaN,NaN,NaN,NaN,NaN
992,Granite Falls Health Center,Incision & Drainage Pilonidal Cyst Simple,NaN,62.6262,62.6262,NaN,62.6262,NaN,NaN,NaN,NaN,NaN
993,Granite Falls Health Center,Incision & Drainage Abscess Complicated/Multiple,NaN,329.7904,329.7904,NaN,329.7904,NaN,NaN,NaN,NaN,NaN


In [7]:
def payer_category_from_payer(payer):
    if payer == 'Gross Charge':
        return 'gross'
    elif payer == 'Discounted Cash Price':
        return 'cash'
    elif payer.startswith('Min'):
        return 'min'
    elif payer.startswith('Max'):
        return 'max'
    
    return 'payer'

def convert_bundle_dataframe(df_in, ccn):
    df_mid = pd.DataFrame(df_in)
    
    df_mid.columns = list(map(lambda c: c.strip(), df_mid.columns.to_list()))
    
    df_mid = df_mid.rename(columns={
        'Item / Service Description': 'description',
        'Item /Service Description': 'description',
        'Patient Type': 'setting'
    })
    
    del df_mid['Facility Name']
    
    if 'setting' in df_mid.columns.to_list():
        df_mid.loc[df_mid['setting'] == 'O', 'setting'] = 'outpatient'
        df_mid.loc[df_mid['setting'] == 'I', 'setting'] = 'inpatient'
        
    money_columns = df_mid.columns[2:]
    remaining_columns = df_mid.columns[:2]
    
    df_mid = pd.melt(df_mid, id_vars=remaining_columns, 
                     var_name='payer_name', value_name='standard_charge')
        
    df_mid['payer_category'] = df_mid['payer_name'].apply(payer_category_from_payer)
        
    df_mid['hospital_id'] = ccn
    df_mid['line_type'] = None
    df_mid['rev_code'] = None
    df_mid['local_code'] = None
    df_mid['code'] = None
    df_mid['ms_drg'] = None
    df_mid['apr_drg'] = None
    df_mid['eapg'] = None
    df_mid['hcpcs_cpt'] = None
    df_mid['modifiers'] = None
    df_mid['thru'] = None
    df_mid['apc'] = None
    df_mid['icd'] = None
    df_mid['ndc'] = None
    df_mid['drug_hcpcs_multiplier'] = None
    df_mid['drug_quantity'] = None
    df_mid['drug_unit_of_measurement'] = None
    df_mid['drug_type_of_measurement'] = None
    df_mid['billing_class'] = None
    df_mid['plan_name'] = None
    df_mid['standard_charge_percent'] = None
    df_mid['contracting_method'] = 'other'
    df_mid['additional_generic_notes'] = 'Service Bundle'
    df_mid['additional_payer_specific_notes'] = None
    
    if not 'setting' in df_mid.columns.to_list():
        df_mid['setting'] = None
        
    df_mid = df_mid[df_mid['standard_charge'].notnull()]
    
    df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
    
    return df_out

In [8]:
df_out = convert_bundle_dataframe(df_in, ccn)
df_out

                   Facility Name   
0    Granite Falls Health Center  \
1    Granite Falls Health Center   
2    Granite Falls Health Center   
3    Granite Falls Health Center   
4    Granite Falls Health Center   
..                           ...   
990  Granite Falls Health Center   
991  Granite Falls Health Center   
992  Granite Falls Health Center   
993  Granite Falls Health Center   
994  Granite Falls Health Center   

                           Item / Service Description  Gross Charge   
0         Tobacco Use Cessation Intensive >10 Minutes           NaN  \
1     Tobacco Use Cessation Intermediate 3-10 Minutes           NaN   
2     Prolng E/M Before&/After Dir Care Ea 30 Minutes           NaN   
3     Prolng E/M Svc Before&/After Dir Pt Care 1St Hr           NaN   
4    Nursing Facility Discharge Management 30 Minutes           NaN   
..                                                ...           ...   
990   Incision & Removal Foreign Body Subq Tiss Compl           NaN   

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,241343,None,Tobacco Use Cessation Intensive >10 Minutes,None,None,None,None,None,None,None,...,None,None,min,Minimum Expected Payment,None,37.7270,None,other,Service Bundle,None
1,241343,None,Tobacco Use Cessation Intermediate 3-10 Minutes,None,None,None,None,None,None,None,...,None,None,min,Minimum Expected Payment,None,19.3502,None,other,Service Bundle,None
2,241343,None,Prolng E/M Before&/After Dir Care Ea 30 Minutes,None,None,None,None,None,None,None,...,None,None,min,Minimum Expected Payment,None,171.1820,None,other,Service Bundle,None
3,241343,None,Prolng E/M Svc Before&/After Dir Pt Care 1St Hr,None,None,None,None,None,None,None,...,None,None,min,Minimum Expected Payment,None,332.9958,None,other,Service Bundle,None
4,241343,None,Nursing Facility Discharge Management 30 Minutes,None,None,None,None,None,None,None,...,None,None,min,Minimum Expected Payment,None,307.8300,None,other,Service Bundle,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,241343,None,Incision & Removal Foreign Body Subq Tiss Compl,None,None,None,None,None,None,None,...,None,None,payer,BCBSMN AWA Expected Payment,None,438.0300,None,other,Service Bundle,None
3976,241343,None,Incision & Removal Foreign Body Subq Tiss Simple,None,None,None,None,None,None,None,...,None,None,payer,BCBSMN AWA Expected Payment,None,242.4138,None,other,Service Bundle,None
3977,241343,None,Incision & Drainage Pilonidal Cyst Simple,None,None,None,None,None,None,None,...,None,None,payer,BCBSMN AWA Expected Payment,None,62.6262,None,other,Service Bundle,None
3978,241343,None,Incision & Drainage Abscess Complicated/Multiple,None,None,None,None,None,None,None,...,None,None,payer,BCBSMN AWA Expected Payment,None,329.7904,None,other,Service Bundle,None


In [9]:
set(df_out['payer_name'].to_list())

{'BCBSMN AWA Expected Payment',
 'Maximum Expected Payment',
 'Minimum Expected Payment'}